In [3]:
# Enhanced Bangla Clinical Information Extraction from Electronic Health Records (EHRs)
# Complete Implementation Protocol and Framework with All Requested Modifications
"""
# Enhanced Bangla Clinical Information Extraction Protocol
## Key Enhancements:
1. Improved patient name extraction (proper Bengali names only)
2. Added weight (kg) and gender (male/female) extraction
3. Enhanced diagnostic test extraction
4. Comprehensive disease entity recognition
5. Added prior medical history extraction
6. Clear separation between condition duration and medication duration
7. Improved symptom extraction
8. Enhanced medication detail extraction
"""

# ==========================================
# SECTION 1: INSTALLATION AND IMPORTS
# ==========================================
import subprocess
import sys

def install_packages():
    packages = [
        'transformers', 'torch', 'pandas', 'numpy', 'scikit-learn',
        'datasets', 'seqeval', 'plotly', 'matplotlib', 'seaborn'
    ]
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

install_packages()

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import re
import json
import hashlib
from datetime import datetime
from typing import List, Dict, Tuple, Optional, Any
import logging
from sklearn.metrics import precision_recall_fscore_support, classification_report
import unicodedata
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import time

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
print("All packages installed and imported successfully!")

# ==========================================
# SECTION 2: DATA COLLECTION PROTOCOL
# ==========================================
class BanglaClinicalDataCollector:
    """
    Enhanced protocol for collecting and organizing Bangla clinical data from various sources
    """
    def __init__(self):
        self.data_sources = {
            'hospital_ehr': 'Electronic Health Records from hospitals',
            'clinic_notes': 'Clinical notes from private clinics',
            'discharge_summaries': 'Patient discharge summaries',
            'prescription_notes': 'Prescription and medication records',
            'lab_reports': 'Laboratory test reports with interpretations',
            'consultation_notes': 'Doctor-patient consultation records'
        }
        self.data_types = {
            'patient_name': 'Patient identification',
            'age': 'Patient age information',
            'gender': 'Patient gender (male/female)',
            'weight': 'Patient weight in kg',
            'chief_complaint': 'Primary reason for visit',
            'history_present_illness': 'Current illness description',
            'past_medical_history': 'Previous medical conditions',
            'medications': 'Current and prescribed medications',
            'physical_examination': 'Physical exam findings',
            'assessment_plan': 'Diagnosis and treatment plan',
            'diagnostic_tests': 'Laboratory and imaging tests'
        }

    def create_data_collection_guidelines(self):
        """Create comprehensive data collection guidelines"""
        guidelines = {
            'inclusion_criteria': [
                'Native Bangla clinical text',
                'Complete patient records with all sections',
                'Verified by medical professionals',
                'Covers diverse medical specialties',
                'Includes various complexity levels',
                'Contains proper patient demographics'
            ],
            'exclusion_criteria': [
                'Incomplete or corrupted records',
                'Non-clinical administrative text',
                'Heavily code-mixed text (>50% English)',
                'Records with patient identifying information',
                'Records without essential clinical information'
            ],
            'quality_standards': [
                'Minimum 500 characters per document',
                'Clear, readable handwriting (if scanned)',
                'Proper sentence structure',
                'Medical terminology consistency',
                'Temporal information included',
                'Complete demographic information'
            ],
            'annotation_guidelines': [
                'BIO tagging scheme for entities',
                'Standardized entity type definitions',
                'Inter-annotator agreement >0.8',
                'Medical expert validation',
                'Quality control checkpoints',
                'Separate annotation for condition vs medication duration'
            ]
        }
        return guidelines

# ==========================================
# SECTION 3: ENHANCED TEXT PREPROCESSING AND NORMALIZATION
# ==========================================
class BanglaTextNormalizer:
    """
    Enhanced comprehensive text normalization for Bangla clinical text
    """
    def __init__(self):
        self.bangla_to_arabic_digits = str.maketrans('০১২৩৪৫৬৭৮৯', '0123456789')
        self.clinical_abbreviations = {
            'ডাঃ': 'ডাক্তার',
            'রাঃ': 'রোগী',
            'সিসিঃ': 'প্রধান অভিযোগ',
            'এইচপিআইঃ': 'বর্তমান অসুস্থতার ইতিহাস',
            'পিএমএইচঃ': 'পূর্ববর্তী চিকিৎসা ইতিহাস',
            'পিইঃ': 'শারীরিক পরীক্ষা',
            'এ/পিঃ': 'মূল্যায়ন ও পরিকল্পনা',
            'কেজিঃ': 'কেজি',
            'কিলোগ্রামঃ': 'কিলোগ্রাম'
        }

        # Enhanced medical term standardization
        self.medical_variants = {
            'জ্বর': ['জর', 'জ্বর', 'জ্বড়', 'জরে'],
            'মাথাব্যথা': ['মাথা ব্যথা', 'মাথাব্যাথা', 'মাথার ব্যথা', 'মাথা যন্ত্রণা', 'মাথাব্যথায়'],
            'কাশি': ['কাশি','কাসি', 'কাশী', 'কাসী', 'কাশ', 'কাশিতে'],
            'শ্বাসকষ্ট': ['শাসকষ্ট', 'শ্বাস কষ্ট', 'নিঃশ্বাস কষ্ট', 'দম বন্ধ', 'দমবন্ধ'],
            'পেটব্যথা': ['পেট ব্যথা', 'পেটে ব্যথা', 'পেটব্যথায়', 'উদর ব্যথা', 'পেট যন্ত্রণা', 'পেটে যন্ত্রণা'],
            'বমি': ['বমী', 'বমন', 'বমিভাব', 'বমির'],
            'ডায়াবেটিস': ['ডায়েবেটিস', 'ডায়াবেটিস', 'ডাইবেটিস','বহুমূত্র', 'মধুমেহ', 'ডায়াবেটিক','ডাইবেটিসে','ডায়াবেটিসে', 'ডায়াবেটিসের'],
            'উচ্চরক্তচাপ': ['হাই প্রেশার', 'উচ্চরক্তচাপ', 'উচ্চ রক্তচাপ', 'উচ্চ রক্ত চাপ', 'হাইপারটেনশন', 'উচ্চ রক্তচাপে', 'উচ্চরক্তচাপে'],
            'হৃদরোগ': ['হার্টের রোগ', 'হৃদয়ের রোগ', 'কার্ডিয়াক', 'হৃদরোগে']
        }

    def normalize_unicode(self, text: str) -> str:
        """Normalize Unicode representations to standard form"""
        return unicodedata.normalize('NFC', text)

    def standardize_digits(self, text: str) -> str:
        """Convert Bangla digits to Arabic numerals"""
        return text.translate(self.bangla_to_arabic_digits)

    def expand_abbreviations(self, text: str) -> str:
        """Expand common clinical abbreviations"""
        for abbrev, expansion in self.clinical_abbreviations.items():
            text = text.replace(abbrev, expansion)
        return text

    def standardize_medical_terms(self, text: str) -> str:
        """Standardize medical term variations"""
        for standard, variants in self.medical_variants.items():
            for variant in variants:
                text = text.replace(variant, standard)
        return text

    def remove_noise(self, text: str) -> str:
        """Remove irrelevant characters while preserving clinical punctuation"""
        # Remove excessive whitespace
        text = re.sub(r'\s+', ' ', text)
        # Keep Bangla characters, digits, and essential punctuation
        text = re.sub(r'[^\u0980-\u09FF\s\.\,\:\;\-\(\)\d\+\=\/\%]', '', text)
        return text.strip()

    def process(self, text: str) -> str:
        """Complete normalization pipeline"""
        if not text:
            return ""
        text = self.normalize_unicode(text)
        text = self.expand_abbreviations(text)
        text = self.standardize_medical_terms(text)
        text = self.standardize_digits(text)
        text = self.remove_noise(text)
        return text

# ==========================================
# SECTION 4: ENHANCED CLINICAL TOKENIZATION
# ==========================================
class BanglaClinicalTokenizer:
    """
    Enhanced specialized tokenizer for Bangla clinical text
    """
    def __init__(self):
        self.medical_compounds = {
            'রক্ত চাপ': 'রক্তচাপ',
            'হৃদ রোগ': 'হৃদরোগ',
            'চোখ ব্যথা': 'চোখব্যথা',
            'পেট ব্যথা': 'পেটব্যথা',
            'মাথা ব্যথা': 'মাথাব্যথা',
            'রক্ত পরীক্ষা': 'রক্ত_পরীক্ষা',
            'প্রস্রাব পরীক্ষা': 'প্রস্রাব_পরীক্ষা',
            'এক্স রে': 'এক্স_রে'
        }

        # Medical measurement patterns
        self.measurement_patterns = [
            r'\d+\.?\d*\s*(?:মিলিগ্রাম|গ্রাম|মিলি|ট্যাবলেট|ক্যাপসুল |ইউনিট)',
            r'\d+\.?\d*\s*(?:ডিগ্রি|°)',
            r'\d+\/\d+\s*(?:mmHg|মিমি)'
        ]

    def segment_sentences(self, text: str) -> List[str]:
        """Clinical-aware sentence segmentation"""
        # Protect decimal numbers and measurements
        text = re.sub(r'(\d+)\.(\d+)', r'\1DECIMAL\2', text)
        # Protect medical abbreviations
        text = re.sub(r'(ডাঃ|রাঃ|সিসিঃ)', r'\1ABBREV', text)
        # Split on sentence boundaries
        sentences = re.split(r'[।\.!?]+', text)
        # Restore protected elements and clean
        sentences = [s.replace('DECIMAL', '.').replace('ABBREV', '').strip()
                    for s in sentences if s.strip()]
        return sentences

    def tokenize_words(self, text: str) -> List[str]:
        """Word tokenization with medical compound handling"""
        # Handle medical compounds first
        for compound, merged in self.medical_compounds.items():
            text = text.replace(compound, merged)

        # Protect measurements
        for pattern in self.measurement_patterns:
            text = re.sub(pattern, lambda m: m.group().replace(' ', '_SPACE_'), text)

        # Basic word tokenization
        tokens = re.findall(r'\S+', text)

        # Restore spaces in measurements
        tokens = [token.replace('_SPACE_', ' ') for token in tokens]
        return tokens

# ==========================================
# SECTION 5: ENHANCED ENTITY RECOGNITION SYSTEM
# ==========================================
class EnhancedClinicalEntityRecognizer:
    """
    Enhanced rule-based and dictionary-based clinical entity recognition
    """
    def __init__(self, lexicon_path: Optional[str] = None):
        self.entity_patterns = self._compile_patterns()
        self.lexicons = self._load_lexicons(lexicon_path) if lexicon_path else self._enhanced_lexicons()
        self.name_patterns = self._compile_name_patterns()

    def _enhanced_lexicons(self) -> Dict[str, set]:
        """Enhanced medical lexicons for Bangla clinical terms"""
        return {
            'symptoms': {
                'জ্বর', 'মাথাব্যথা', 'মাথাব্যথায়' ' কাশি', 'শ্বাসকষ্ট', 'বমি', 'ডায়রিয়া', 'পেটব্যথা',
                'চোখব্যথা', 'গলাব্যথা', 'অরুচি', 'দুর্বলতা', 'মাথাঘোরা', 'বুকব্যথা',
                'হাঁপানি', 'কোষ্ঠকাঠিন্য', 'অনিদ্রা', 'ক্লান্তি', 'বুক ধড়ফড়ানি',
                'পিঠব্যথা', 'কানব্যথা', 'দাঁতব্যথা', 'জয়েন্টে ব্যথা', 'পায়ে ব্যথা',
                'হাতে ব্যথা', 'ঘাড়ব্যথা', 'কোমর ব্যথা', 'বুক জ্বালা', 'গ্যাস',
                'এসিডিটি', 'বদহজম', 'মুখ শুকিয়ে যাওয়া', 'প্রস্রাবে জ্বালা', 'বুক ধড়ফড়ানি'
            },
            'diseases': {
                'ডায়াবেটিস', 'হৃদরোগ', 'উচ্চ রক্তচাপ', 'যক্ষ্মা', 'ক্যান্সার', 'অ্যাজমা',
                'আর্থ্রাইটিস', 'হেপাটাইটিস', 'ম্যালেরিয়া', 'টাইফয়েড', 'নিউমোনিয়া',
                'ব্রংকাইটিস', 'গ্যাস্ট্রাইটিস', 'আলসার', 'কিডনি রোগ', 'লিভার রোগ',
                'থাইরয়েড', 'মাইগ্রেন', 'সাইনাস', 'টনসিল', 'গলব্লাডার স্টোন',
                'কিডনি স্টোন', 'অ্যাপেন্ডিসাইটিস', 'কলেরা', 'ডেঙ্গু', 'চিকনগুনিয়া',
                'ভাইরাল ফিভার', 'ইনফেকশন', 'এলার্জি', 'একজিমা', 'সোরিয়াসিস'
            },
            'medications': {
                'প্যারাসিটামল', 'ইনসুলিন', 'অ্যান্টিবায়োটিক', 'অ্যান্টাসিড', 'আইবুপ্রোফেন',
                'মেটফরমিন', 'অ্যামক্সিসিলিন', 'ওমিপ্রাজল', 'এসপিরিন', 'প্রেডনিসোলন',
                'ফুরোসেমাইড', 'এটোরভাস্ট্যাটিন', 'লসার্টান', 'এমলোডিপিন', 'র্যানিটিডিন',
                'ডোমপেরিডন', 'সিপ্রোফ্লক্সাসিন', 'ডক্সিসাইক্লিন', 'সেফট্রিয়াক্সন', 'কুইনাপ্রিল',
                'নাপা', 'স্কয়ার', 'বেক্সিমকো', 'এসকে ফার্মা', 'রেনাটা', 'ইনসেপ্টা'
            },
            'body_parts': {
                'মাথা', 'চোখ', 'নাক', 'গলা', 'বুক', 'পেট', 'হাত', 'পা', 'হৃদয়',
                'ফুসফুস', 'কিডনি', 'লিভার', 'মস্তিষ্ক', 'হাড়', 'জয়েন্ট', 'মেরুদণ্ড',
                'কান', 'দাঁত', 'জিহ্বা', 'গলা', 'ঘাড়', 'কাঁধ', 'কনুই', 'কব্জি',
                'পিঠ', 'কোমর', 'নিতম্ব', 'জানু', 'গোড়ালি', 'পায়ের আঙুল', 'হাতের আঙুল'
            },
            'tests': {
                'রক্ত পরীক্ষা', 'প্রস্রাব পরীক্ষা', 'এক্স রে', 'ইসিজি', 'আল্ট্রাসাউন্ড',
                'সিটি স্ক্যান', 'এমআরআই', 'এন্ডোস্কোপি', 'কলোনস্কোপি', 'ইকো',
                'স্ট্রেস টেস্ট', 'গ্লুকোজ টেস্ট', 'এইচবিএ ওয়ানসি', 'লিপিড প্রোফাইল',
                'থাইরয়েড ফাংশন টেস্ট', 'লিভার ফাংশন টেস্ট', 'কিডনি ফাংশন টেস্ট',
                'বায়োপসি', 'মেমোগ্রাম', 'প্যাপ স্মিয়ার', 'স্পুটাম টেস্ট'
            }
        }

    def _load_lexicons(self, lexicon_path: str) -> Dict[str, set]:
        """Load lexicons from JSON file"""
        try:
            with open(lexicon_path, 'r', encoding='utf-8') as f:
                lexicons_data = json.load(f)
            return {key: set(value) for key, value in lexicons_data.items()}
        except FileNotFoundError:
            logger.warning(f"Lexicon file not found at {lexicon_path}. Using default lexicons.")
            return self._enhanced_lexicons()
        except json.JSONDecodeError:
            logger.warning(f"Error decoding lexicon JSON from {lexicon_path}. Using default lexicons.")
            return self._enhanced_lexicons()
        except Exception as e:
            logger.error(f"An error occurred loading lexicons: {e}. Using default lexicons.")
            return self._enhanced_lexicons()


    def _compile_patterns(self) -> Dict[str, re.Pattern]:
        """Compile regex patterns for clinical entities"""
        return {
            # Enhanced patient name patterns - more specific for Bengali names
            # Corrected pattern for Bengali characters using Unicode range
            'patient_name': re.compile(r'(?:রোগী[ঃ:]?\s*|নাম[ঃ:]?\s*|Patient[ঃ:]?\s*)([\u0980-\u09FF\s]{1,25})', re.IGNORECASE),
            'alternative_name': re.compile(r'([\u0980-\u09FF\s]{2,20})\s*(?:নামে|নামের)', re.IGNORECASE),

            # Enhanced dosage patterns
            'dosage': re.compile(r'(\d+\.?\d*)\s*(মিলিগ্রাম|গ্রাম|মিলি|ট্যাবলেট|ক্যাপসুল|ইউনিট)', re.IGNORECASE),

            # Enhanced frequency patterns
            'frequency': re.compile(r'(দিনে|সকালে|বিকালে|রাতে|প্রতিদিন)\s*(\d+)\s*(বার|টি)|(\d+)\s*বার\s*(দিনে)', re.IGNORECASE),

            # Medication duration patterns (specific)
            'medication_duration': re.compile(r'(\d+)\s*(দিন|সপ্তাহ|মাস)\s*(?:খেতে|সেবন|চালিয়ে|খাবেন)', re.IGNORECASE),

            # Condition duration patterns (different from medication)
            'condition_duration': re.compile(r'(?:গত|বিগত|প্রায়)\s*(\d+)\s*(দিন|সপ্তাহ|মাস|বছর)\s*(?:ধরে|যাবত|থেকে)', re.IGNORECASE),

            # Enhanced vital sign patterns
            'temperature': re.compile(r'তাপমাত্রা[ঃ:]?\s*(\d+\.?\d*)\s*(ডিগ্রি|°)', re.IGNORECASE),
            'blood_pressure': re.compile(r'রক্তচাপ[ঃ:]?\s*(\d+)\/(\d+)', re.IGNORECASE),

            # Enhanced demographic patterns
            'age': re.compile(r'(\d+)\s*বছর\s*বয়সী', re.IGNORECASE),
            'weight': re.compile(r'ওজন[ঃ:]?\s*(\d+\.?\d*)\s*(?:কেজি|কিলোগ্রাম|kg)', re.IGNORECASE),
            'alternative_weight': re.compile(r'(\d+\.?\d*)\s*(?:কেজি|কিলোগ্রাম|kg)', re.IGNORECASE),

            # Gender patterns
            'gender': re.compile(r'(পুরুষ|মহিলা|নারী|ছেলে|মেয়ে|male|female)', re.IGNORECASE),

            # Prior history patterns
            'prior_history': re.compile(r'(?:পূর্ববর্তী|আগের|অতীতে|ইতিহাস)\s*([^।\.]+)', re.IGNORECASE)
        }

    def _compile_name_patterns(self) -> List[re.Pattern]:
        """Compile patterns for Bengali name recognition"""
        return [
            # Common Bengali name patterns
            re.compile(r'\b([\u0980-\u09FF]{1,15}(?:\s+[\u0980-\u09FF]{1,15}){0,2})\b'),
            # Names with titles
            re.compile(r'(?:জনাব|মিসেস|মিস)\s+([\u0980-\u09FF\s]{2,25})'),
        ]

    def extract_entities(self, text: str) -> Dict[str, List[Tuple[str, int, int]]]:
        """Enhanced extract clinical entities with positions"""
        entities = {
            'PATIENT_NAME': [], 'SYMPTOM': [], 'DISEASE': [], 'MEDICATION': [], 'BODY_PART': [],
            'DOSAGE': [], 'FREQUENCY': [], 'MEDICATION_DURATION': [], 'CONDITION_DURATION': [],
            'VITAL_SIGN': [], 'TEST': [], 'AGE': [], 'WEIGHT': [], 'GENDER': [], 'PRIOR_HISTORY': []
        }

        # Enhanced patient name extraction
        self._extract_patient_names(text, entities)

        # Enhanced dictionary-based extraction
        self._extract_dictionary_entities(text, entities)

        # Enhanced pattern-based extraction
        self._extract_pattern_entities(text, entities)

        return entities

    def _extract_patient_names(self, text: str, entities: Dict):
        """Extract patient names more accurately"""
        # Try main pattern first
        name_match = self.entity_patterns['patient_name'].search(text)
        if name_match:
            name = name_match.group(1).strip()
            # Validate it's actually a name (not a symptom or other text)
            if self._is_valid_name(name):
                entities['PATIENT_NAME'].append((name, name_match.start(1), name_match.end(1)))

        # Try alternative pattern
        if not entities['PATIENT_NAME']:
            alt_match = self.entity_patterns['alternative_name'].search(text)
            if alt_match:
                name = alt_match.group(1).strip()
                if self._is_valid_name(name):
                    entities['PATIENT_NAME'].append((name, alt_match.start(1), alt_match.end(1)))

    def _is_valid_name(self, candidate: str) -> bool:
        """Validate if a candidate string is actually a name"""
        # Check if it's not a symptom, disease, or medication
        all_medical_terms = set()
        for category in ['symptoms', 'diseases', 'medications']:
            all_medical_terms.update(self.lexicons[category])

        # Clean candidate for comparison
        candidate_clean = candidate.strip().lower()

        # Reject if it's a known medical term
        if candidate_clean in all_medical_terms:
            return False

        # Reject if it contains common non-name words
        non_name_words = {'ভুগেছন', 'করেছেন', 'হয়েছে', 'আছে', 'নেই', 'করুন', 'খান'}
        if any(word in candidate_clean for word in non_name_words):
            return False

        # Accept if it looks like a proper Bengali name
        if re.match(r'^[\u0980-\u09FF\s]{1,25}$', candidate) and len(candidate.split()) <= 3:
            return True

        return False

    def _extract_dictionary_entities(self, text: str, entities: Dict):
        """Extract entities using enhanced dictionary matching"""
        words = text.split()
        current_pos = 0

        for i, word in enumerate(words):
            word_clean = re.sub(r'[^\u0980-\u09FF]', '', word)

            # Find position in original text
            start_pos = text.find(word, current_pos)
            end_pos = start_pos + len(word)
            current_pos = end_pos

            # Check for multi-word medical terms first
            if i < len(words) - 1:
                next_word_clean = re.sub(r'[^\u0980-\u09FF]', '', words[i+1])
                two_word = f"{word_clean} {next_word_clean}"
                if self._check_and_add_entity(two_word, entities, start_pos, text.find(words[i+1], end_pos) + len(words[i+1])):
                    # Advance current_pos by the length of the second word as well
                    current_pos += len(words[i+1])
                    continue

            # Single word matching
            self._check_and_add_entity(word_clean, entities, start_pos, end_pos)


    def _check_and_add_entity(self, term: str, entities: Dict, start_pos: int, end_pos: int) -> bool:
        """Check if term matches any lexicon and add to entities"""
        for category, entity_type in [('symptoms', 'SYMPTOM'), ('diseases', 'DISEASE'),
                                     ('medications', 'MEDICATION'), ('body_parts', 'BODY_PART'),
                                     ('tests', 'TEST')]:
            if term in self.lexicons[category]:
                entities[entity_type].append((term, start_pos, end_pos))
                return True
        return False

    def _extract_pattern_entities(self, text: str, entities: Dict):
        """Extract entities using enhanced pattern matching"""
        pattern_mappings = {
            'dosage': 'DOSAGE',
            'frequency': 'FREQUENCY',
            'medication_duration': 'MEDICATION_DURATION',
            'condition_duration': 'CONDITION_DURATION',
            'temperature': 'VITAL_SIGN',
            'blood_pressure': 'VITAL_SIGN',
            'age': 'AGE',
            'weight': 'WEIGHT',
            'alternative_weight': 'WEIGHT',
            'gender': 'GENDER',
            'prior_history': 'PRIOR_HISTORY'
        }

        for pattern_name, entity_type in pattern_mappings.items():
            pattern = self.entity_patterns[pattern_name]
            for match in pattern.finditer(text):
                if pattern_name in ['temperature', 'blood_pressure']:
                    entities[entity_type].append((match.group(), match.start(), match.end()))
                elif pattern_name == 'weight' or pattern_name == 'alternative_weight':
                    weight_value = f"{match.group(1)} কেজি"
                    entities[entity_type].append((weight_value, match.start(), match.end()))
                elif pattern_name == 'gender':
                    gender_bengali = self._standardize_gender(match.group(1))
                    entities[entity_type].append((gender_bengali, match.start(), match.end()))
                else:
                    entities[entity_type].append((match.group(), match.start(), match.end()))

    def _standardize_gender(self, gender_text: str) -> str:
        """Standardize gender to Bengali male/female"""
        male_terms = ['পুরুষ', 'ছেলে', 'male']
        female_terms = ['মহিলা', 'নারী', 'মেয়ে', 'female']

        gender_lower = gender_text.lower()
        if any(term in gender_lower for term in male_terms):
            return 'পুরুষ'
        elif any(term in gender_lower for term in female_terms):
            return 'মহিলা'
        return gender_text

# ==========================================
# SECTION 6: ENHANCED RELATION EXTRACTION
# ==========================================
class EnhancedClinicalRelationExtractor:
    """
    Enhanced extract relationships between clinical entities with better separation of durations
    """
    def __init__(self):
        self.relation_patterns = self._compile_relation_patterns()
        self.temporal_patterns = self._compile_temporal_patterns()
        self.medication_patterns = self._compile_medication_patterns()

    def _compile_relation_patterns(self) -> Dict[str, List[re.Pattern]]:
        """Compile enhanced regex patterns for clinical relationships"""
        return {
            'symptom_of': [
                re.compile(r'(.+?)\s+(এর|র)\s+লক্ষণ', re.IGNORECASE),
                re.compile(r'(.+?)\s+থেকে\s+(.+?)\s+হয়েছে', re.IGNORECASE),
                re.compile(r'(.+?)\s+কারণে\s+(.+?)', re.IGNORECASE),
                re.compile(r'(.+?)\s+জন্য\s+(.+?)', re.IGNORECASE)
            ],
            'treats': [
                re.compile(r'(.+?)\s+দিয়ে\s+(.+?)\s+চিকিৎসা', re.IGNORECASE),
                re.compile(r'(.+?)\s+খেলে\s+(.+?)\s+ভাল', re.IGNORECASE),
                re.compile(r'(.+?)\s+ওষুধ\s+(.+?)\s+জন্য', re.IGNORECASE),
                re.compile(r'(.+?)\s+সেবন\s+করুন\s+(.+?)', re.IGNORECASE)
            ],
            'located_in': [
                re.compile(r'(.+?)\s+(.+?)\s+অংশে', re.IGNORECASE),
                re.compile(r'(.+?)\s+(.+?)\s+এলাকায়', re.IGNORECASE),
                re.compile(r'(.+?)\s+(.+?)\s+স্থানে', re.IGNORECASE)
            ],
            'has_dosage': [
                re.compile(r'(.+?)\s+(\d+\.?\d*)\s*(মিলিগ্রাম|গ্রাম|মিলি|ইউনিট)', re.IGNORECASE)
            ],
            'has_frequency': [
                re.compile(r'(.+?)\s+(দিনে|সকালে|বিকালে)\s*(\d+)\s*বার', re.IGNORECASE),
                re.compile(r'(.+?)\s+(\d+)\s*বার\s*(দিনে)', re.IGNORECASE)
            ],
            'condition_has_duration': [
                re.compile(r'(.+?)\s+(?:গত|বিগত|প্রায়)\s*(\d+)\s*(দিন|সপ্তাহ|মাস|বছর)\s*(?:ধরে|যাবত|থেকে)', re.IGNORECASE)
            ],
            'medication_has_duration': [
                re.compile(r'(.+?)\s+(\d+)\s*(দিন|সপ্তাহ|মাস)\s*(?:খেতে|সেবন|চালিয়ে)', re.IGNORECASE)
            ]
        }

    def _compile_temporal_patterns(self) -> Dict[str, re.Pattern]:
        """Compile enhanced temporal relationship patterns"""
        return {
            'before': re.compile(r'(.+?)\s+(আগে|পূর্বে)\s+(.+?)', re.IGNORECASE),
            'after': re.compile(r'(.+?)\s+(পরে|পর)\s+(.+?)', re.IGNORECASE),
            'during': re.compile(r'(.+?)\s+(সময়|মধ্যে)\s+(.+?)', re.IGNORECASE),
            'since': re.compile(r'(.+?)\s+থেকে\s+(.+?)', re.IGNORECASE),
            'for_duration': re.compile(r'(.+?)\s+(গত|বিগত)\s+(\d+)\s+(দিন|সপ্তাহ|মাস|বছর)', re.IGNORECASE)
        }

    def _compile_medication_patterns(self) -> Dict[str, re.Pattern]:
        """Compile specific medication relationship patterns"""
        return {
            'medication_with_dosage': re.compile(r'([\u0980-\u09FF\s]+)\s+(\d+\.?\d*)\s*(মিলিগ্রাম|গ্রাম|মিলি|ইউনিট)', re.IGNORECASE),
            'medication_with_frequency': re.compile(r'([\u0980-\u09FF\s]+)\s+(?:দিনে|প্রতিদিন)\s*(\d+)\s*বার', re.IGNORECASE),
            'medication_with_duration': re.compile(r'([\u0980-\u09FF\s]+)\s+(\d+)\s*(দিন|সপ্তাহ|মাস)\s*(?:খেতে|সেবন)', re.IGNORECASE)
        }

    def extract_relations(self, text: str, entities: Dict) -> List[Dict]:
        """Extract enhanced relationships between entities"""
        relations = []

        # Extract pattern-based relations
        for relation_type, patterns in self.relation_patterns.items():
            for pattern in patterns:
                matches = pattern.finditer(text)
                for match in matches:
                    groups = match.groups()
                    if len(groups) >= 2:
                        relations.append({
                            'relation': relation_type,
                            'arg1': groups[0].strip(),
                            'arg2': groups[1].strip() if len(groups) > 1 else groups[0].strip(),
                            'confidence': self._calculate_confidence(match, entities),
                            'start': match.start(),
                            'end': match.end(),
                            'pattern_type': 'clinical'
                        })

        # Extract temporal relations
        for relation_type, pattern in self.temporal_patterns.items():
            matches = pattern.finditer(text)
            for match in matches:
                groups = match.groups()
                if len(groups) >= 2:
                    relations.append({
                        'relation': f'temporal_{relation_type}',
                        'arg1': groups[0].strip(),
                        'arg2': groups[2].strip() if len(groups) > 2 else groups[1].strip(),
                        'confidence': self._calculate_confidence(match, entities),
                        'start': match.start(),
                        'end': match.end(),
                        'pattern_type': 'temporal'
                    })

        return relations

    def _calculate_confidence(self, match: re.Match, entities: Dict) -> float:
        """Calculate confidence score for relation"""
        matched_text = match.group()
        entity_count = sum(
            1 for entity_list in entities.values()
            for entity, _, _ in entity_list
            if entity in matched_text
        )
        return min(entity_count / 2.0, 1.0)

    def extract_medication_relations(self, text: str, entities: Dict) -> List[Dict]:
        """Extract enhanced medication-related relationships"""
        medication_relations = []
        medications = [e[0] for e in entities.get('MEDICATION', [])]
        dosages = [e[0] for e in entities.get('DOSAGE', [])]
        frequencies = [e[0] for e in entities.get('FREQUENCY', [])]
        med_durations = [e[0] for e in entities.get('MEDICATION_DURATION', [])]

        for med in medications:
            # Find closest dosage
            for dosage in dosages:
                if self._are_close_in_text(text, med, dosage):
                    medication_relations.append({
                        'relation': 'has_dosage',
                        'medication': med,
                        'dosage': dosage,
                        'confidence': 0.9
                    })

            # Find closest frequency
            for freq in frequencies:
                if self._are_close_in_text(text, med, freq):
                    medication_relations.append({
                        'relation': 'has_frequency',
                        'medication': med,
                        'frequency': freq,
                        'confidence': 0.9
                    })

            # Find closest medication duration (not condition duration)
            for dur in med_durations:
                if self._are_close_in_text(text, med, dur):
                    medication_relations.append({
                        'relation': 'has_medication_duration',
                        'medication': med,
                        'duration': dur,
                        'confidence': 0.9
                    })

        return medication_relations

    def _are_close_in_text(self, text: str, entity1: str, entity2: str, max_distance: int = 50) -> bool:
        """Check if two entities are close in text"""
        pos1 = text.find(entity1)
        pos2 = text.find(entity2)
        if pos1 == -1 or pos2 == -1:
            return False
        return abs(pos1 - pos2) <= max_distance

# ==========================================
# SECTION 7: COMPLETE ENHANCED EXTRACTION PIPELINE
# ==========================================
class EnhancedClinicalInformationExtractor:
    """
    Enhanced main class for clinical information extraction pipeline
    """
    def __init__(self, model_path: Optional[str] = None, lexicon_path: Optional[str] = None):
        self.normalizer = BanglaTextNormalizer()
        self.tokenizer = BanglaClinicalTokenizer()
        self.entity_recognizer = EnhancedClinicalEntityRecognizer(lexicon_path)
        self.relation_extractor = EnhancedClinicalRelationExtractor()

        # Cache for performance
        self.cache = {}
        self.stats = {
            'total_requests': 0,
            'cache_hits': 0,
            'processing_times': []
        }

    def extract_clinical_information(self, ehr_text: str) -> Dict[str, Any]:
        """Enhanced main extraction pipeline"""
        start_time = time.time()
        self.stats['total_requests'] += 1

        # Check cache
        text_hash = hashlib.md5(ehr_text.encode()).hexdigest()
        if text_hash in self.cache:
            self.stats['cache_hits'] += 1
            return self.cache[text_hash]

        # Step 1: Normalize text
        normalized_text = self.normalizer.process(ehr_text)

        # Step 2: Extract entities
        entities = self.entity_recognizer.extract_entities(normalized_text)

        # Step 3: Extract relations
        relations = self.relation_extractor.extract_relations(normalized_text, entities)
        medication_relations = self.relation_extractor.extract_medication_relations(
            normalized_text, entities
        )

        # Step 4: Structure information
        structured_info = self._structure_clinical_info(
            entities, relations + medication_relations, normalized_text
        )

        # Cache result and update stats
        processing_time = time.time() - start_time
        self.stats['processing_times'].append(processing_time)
        structured_info['processing_time'] = processing_time
        self.cache[text_hash] = structured_info

        return structured_info

    def _structure_clinical_info(self, entities: Dict, relations: List, text: str) -> Dict[str, Any]:
        """Structure extracted information into enhanced clinical format"""
        # Extract detailed medication information
        medication_details = self._extract_medication_details(entities, relations)

        # Extract patient demographics
        demographics = self._extract_demographics(entities, text)

        # Extract vital signs with values
        vital_signs = self._extract_vital_signs(entities, text)

        # Extract diagnostic tests
        diagnostic_tests = self._extract_diagnostic_tests(entities)

        # Extract prior history
        prior_history = self._extract_prior_history(entities, text)

        return {
            'patient_name': self._extract_patient_name(entities),
            'age': demographics.get('age'),
            'gender': demographics.get('gender'),
            'weight': demographics.get('weight'),
            'symptoms': [e[0] for e in entities.get('SYMPTOM', [])],
            'diseases': [e[0] for e in entities.get('DISEASE', [])],
            'medications': medication_details,
            'duration_of_condition': self._extract_condition_duration(entities, relations),
            'diagnostic_tests': diagnostic_tests,
            'vital_signs': vital_signs,
            'body_parts_mentioned': [e[0] for e in entities.get('BODY_PART', [])],
            'prior_history': prior_history,
            'clinical_relationships': relations,
            'extraction_metadata': {
                'timestamp': datetime.now().isoformat(),
                'text_length': len(text),
                'entity_count': sum(len(ents) for ents in entities.values()),
                'relation_count': len(relations),
                'confidence_score': self._calculate_overall_confidence(entities, relations)
            }
        }

    def _extract_patient_name(self, entities: Dict) -> Optional[str]:
        """Extract patient name"""
        names = entities.get('PATIENT_NAME', [])
        return names[0][0] if names else None

    def _extract_medication_details(self, entities: Dict, relations: List) -> List[Dict]:
        """Extract detailed medication information with dosage, frequency, duration"""
        medications = []
        for med_entity in entities.get('MEDICATION', []):
            med_name = med_entity[0]

            # Initialize medication info
            medication_info = {
                'name': med_name,
                'dosage': None,
                'frequency': None,
                'duration': None  # This is medication duration, not condition duration
            }

            # Find related information from relations
            for relation in relations:
                if (relation.get('medication') == med_name or
                    relation.get('arg1') == med_name):
                    if relation['relation'] == 'has_dosage':
                        medication_info['dosage'] = relation.get('dosage', relation.get('arg2'))
                    elif relation['relation'] == 'has_frequency':
                        medication_info['frequency'] = relation.get('frequency', relation.get('arg2'))
                    elif relation['relation'] == 'has_medication_duration':
                        medication_info['duration'] = relation.get('duration', relation.get('arg2'))

            # If no relations found, look for nearby entities
            if not any([medication_info['dosage'], medication_info['frequency'], medication_info['duration']]):
                # Look for dosage
                for dosage_entity in entities.get('DOSAGE', []):
                    if self._are_entities_related(med_name, dosage_entity[0]):
                        medication_info['dosage'] = dosage_entity[0]
                        break

                # Look for frequency
                for freq_entity in entities.get('FREQUENCY', []):
                    if self._are_entities_related(med_name, freq_entity[0]):
                        medication_info['frequency'] = freq_entity[0]
                        break

                # Look for medication duration only
                for dur_entity in entities.get('MEDICATION_DURATION', []):
                    if self._are_entities_related(med_name, dur_entity[0]):
                        medication_info['duration'] = dur_entity[0]
                        break

            medications.append(medication_info)
        return medications

    def _are_entities_related(self, entity1: str, entity2: str) -> bool:
        """Check if two entities are likely related (simple heuristic)"""
        # This is a simple implementation - in practice, you might want more sophisticated logic
        return True  # For now, assume nearby entities are related

    def _extract_demographics(self, entities: Dict, text: str) -> Dict[str, Any]:
        """Extract enhanced patient demographic information"""
        demographics = {
            'age': None,
            'gender': None,
            'weight': None
        }

        # Extract age
        age_entities = entities.get('AGE', [])
        if age_entities:
            demographics['age'] = age_entities[0][0]

        # Extract weight
        weight_entities = entities.get('WEIGHT', [])
        if weight_entities:
            demographics['weight'] = weight_entities[0][0]

        # Extract gender
        gender_entities = entities.get('GENDER', [])
        if gender_entities:
            demographics['gender'] = gender_entities[0][0]

        return demographics

    def _extract_vital_signs(self, entities: Dict, text: str) -> List[str]:
        """Extract vital signs with specific values"""
        vital_signs = []
        for vital_entity in entities.get('VITAL_SIGN', []):
            vital_signs.append(vital_entity[0])
        return vital_signs

    def _extract_diagnostic_tests(self, entities: Dict) -> List[str]:
        """Extract enhanced diagnostic tests"""
        return [e[0] for e in entities.get('TEST', [])]

    def _extract_prior_history(self, entities: Dict, text: str) -> List[str]:
        """Extract prior medical history"""
        prior_history = []
        for history_entity in entities.get('PRIOR_HISTORY', []):
            prior_history.append(history_entity[0])

        # Also look for history patterns in diseases
        history_keywords = ['ইতিহাস', 'আগে', 'পূর্বে', 'অতীতে']
        for disease in entities.get('DISEASE', []):
            disease_text = disease[0]
            # Check if disease is mentioned with history context
            for keyword in history_keywords:
                if keyword in text and disease_text in text:
                    prior_history.append(f"{disease_text} এর ইতিহাস")
                    break

        return list(set(prior_history))  # Remove duplicates

    def _extract_condition_duration(self, entities: Dict, relations: List) -> Optional[str]:
        """Extract duration of medical condition (separate from medication duration)"""
        condition_durations = entities.get('CONDITION_DURATION', [])

        if condition_durations:
            return condition_durations[0][0]

        # Look in relations for condition duration
        for relation in relations:
            if relation['relation'] == 'condition_has_duration':
                return relation.get('arg2', '')

        return None

    def _calculate_overall_confidence(self, entities: Dict, relations: List) -> float:
        """Calculate overall confidence score for extraction"""
        entity_count = sum(len(ents) for ents in entities.values())
        relation_count = len(relations)

        if entity_count == 0:
            return 0.0

        # Enhanced confidence calculation
        base_confidence = min(entity_count / 10.0, 1.0)
        relation_bonus = min(relation_count / 5.0, 0.2)
        return min(base_confidence + relation_bonus, 1.0)

    def get_stats(self) -> Dict[str, Any]:
        """Get system statistics"""
        avg_processing_time = sum(self.stats['processing_times']) / len(self.stats['processing_times']) if self.stats['processing_times'] else 0
        return {
            'total_requests': self.stats['total_requests'],
            'cache_size': len(self.cache),
            'cache_hit_ratio': self.stats['cache_hits'] / self.stats['total_requests'] if self.stats['total_requests'] > 0 else 0,
            'avg_processing_time': avg_processing_time
        }

# ==========================================
# SECTION 8: ENHANCED EVALUATION FRAMEWORK
# ==========================================
class EnhancedClinicalExtractionEvaluator:
    """
    Enhanced comprehensive evaluation framework for clinical extraction system
    """
    def __init__(self):
        self.metrics = {}

    def evaluate_entity_extraction(self, predicted_entities: Dict, true_entities: Dict) -> Dict[str, Dict[str, float]]:
        """Evaluate entity extraction performance with detailed metrics"""
        results = {}

        for entity_type in set(predicted_entities.keys()) | set(true_entities.keys()):
            # Convert to sets for comparison
            pred_set = set(e[0] if isinstance(e, tuple) else e
                         for e in predicted_entities.get(entity_type, []))
            true_set = set(e[0] if isinstance(e, tuple) else e
                         for e in true_entities.get(entity_type, []))

            if len(pred_set) + len(true_set) == 0:
                continue

            # Calculate metrics
            tp = len(pred_set & true_set)
            fp = len(pred_set - true_set)
            fn = len(true_set - pred_set)

            precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

            results[entity_type] = {
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'true_positives': tp,
                'false_positives': fp,
                'false_negatives': fn,
                'support': len(true_set)
            }

        # Calculate macro average
        if results:
            macro_precision = sum(r['precision'] for r in results.values()) / len(results)
            macro_recall = sum(r['recall'] for r in results.values()) / len(results)
            macro_f1 = sum(r['f1'] for r in results.values()) / len(results)

            results['macro_avg'] = {
                'precision': macro_precision,
                'recall': macro_recall,
                'f1': macro_f1
            }

        return results

    def evaluate_demographic_extraction(self, predicted: Dict, true: Dict) -> Dict[str, float]:
        """Evaluate demographic information extraction accuracy"""
        demographics = ['age', 'gender', 'weight']
        correct = 0
        total = 0

        for demo in demographics:
            if demo in true:
                total += 1
                if demo in predicted and predicted[demo] == true[demo]:
                    correct += 1

        accuracy = correct / total if total > 0 else 0.0
        return {'demographic_accuracy': accuracy, 'correct': correct, 'total': total}

# ==========================================
# SECTION 9: DEMONSTRATION AND TESTING
# ==========================================

# Initialize the complete system
print("Initializing Complete Clinical Information Extraction System...")
extractor = EnhancedClinicalInformationExtractor()
normalizer = BanglaTextNormalizer()
tokenizer = BanglaClinicalTokenizer()
recognizer = EnhancedClinicalEntityRecognizer()
relation_extractor = EnhancedClinicalRelationExtractor()
evaluator = EnhancedClinicalExtractionEvaluator()

print("System initialized successfully!")

# ==========================================
# TEXT NORMALIZATION DEMONSTRATION
# ==========================================

sample_texts = [
    "ডাঃ রহমান রাঃ ৩৫ বছর বয়সী পুরুষ জ্বর ও মাথা ব্যথায় ভুগছেন।",
    "তাপমাত্রা ১০২.৫° ফারেনহাইট। পিE: সাধারণ অবস্থা ভাল।",
    "এ/পি: প্যারাসিটামল ৫০০ মিগ্র দিনে ৩ বার ৫ দিন।"
]

print("\nTEXT NORMALIZATION DEMONSTRATION")
print("=" * 50)
for i, text in enumerate(sample_texts, 1):
    normalized = normalizer.process(text)
    print(f"Original {i}: {text}")
    print(f"Normalized: {normalized}")
    print("-" * 30)

# ==========================================
# TOKENIZATION DEMONSTRATION
# ==========================================

test_text = "রোগী জ্বর ও মাথাব্যথায় ভুগছেন। তাপমাত্রা ১০২.৫ ডিগ্রি। প্যারাসিটামল ৫০০ মিলিগ্রাম দিনে ৩ বার।"
sentences = tokenizer.segment_sentences(test_text)
tokens = tokenizer.tokenize_words(test_text)

print("\nTOKENIZATION DEMONSTRATION")
print("=" * 50)
print(f"Original: {test_text}")
print(f"\nSentences: {sentences}")
print(f"\nTokens: {tokens}")

# ==========================================
# ENTITY RECOGNITION DEMONSTRATION
# ==========================================

sample_clinical_text = """
৩৫ বছর বয়সী পুরুষ রোগী জ্বর ও মাথাব্যথায় ভুগছেন।
তাপমাত্রা ১০২ ডিগ্রি ফারেনহাইট। রক্তচাপ ১৪০/৯০।
প্যারাসিটামল ৫০০ মিলিগ্রাম দিনে ৩ বার ৫ দিন সেবন করুন।
রক্ত পরীক্ষা ও প্রস্রাব পরীক্ষা করাতে হবে।
"""

extracted_entities = recognizer.extract_entities(sample_clinical_text)

print("\nENTITY RECOGNITION DEMONSTRATION")
print("=" * 50)
print(f"Text: {sample_clinical_text.strip()}")
print("\nExtracted Entities:")
for entity_type, entities in extracted_entities.items():
    if entities:
        print(f"{entity_type}: {[e[0] for e in entities]}")

# ==========================================
# RELATION EXTRACTION DEMONSTRATION
# ==========================================

sample_relation_text = """
রোগী জ্বর ও মাথাব্যথায় ভুগছেন। জ্বর থেকে দুর্বলতা হয়েছে।
প্যারাসিটামল ৫০০ মিলিগ্রাম দিনে ৩ বার ৫ দিন সেবন করুন।
ওষুধ খাওয়ার পরে জ্বর কমবে।
"""

# Extract entities first
sample_entities = recognizer.extract_entities(sample_relation_text)
# Extract relations
relations = relation_extractor.extract_relations(sample_relation_text, sample_entities)
medication_relations = relation_extractor.extract_medication_relations(sample_relation_text, sample_entities)

print("\nRELATION EXTRACTION DEMONSTRATION")
print("=" * 50)
print(f"Text: {sample_relation_text.strip()}")
print("\nGeneral Relations:")
for rel in relations:
    if rel['confidence'] > 0.1:  # Filter low confidence relations
        print(f"  {rel['relation']}: {rel['arg1']} → {rel['arg2']} (conf: {rel['confidence']:.2f})")

print("\nMedication Relations:")
for rel in medication_relations:
    print(f"  {rel['relation']}: {rel['medication']} → {rel.get('dosage', rel.get('frequency', rel.get('duration', '')))}")

# ==========================================
# COMPLETE SYSTEM DEMONSTRATION
# ==========================================

print("\n" + "="*70)
print("COMPLETE CLINICAL INFORMATION EXTRACTION DEMONSTRATION")
print("="*70)

# Sample clinical texts from the output document
sample_clinical_texts = [
    "রোগী আব্দুর রহমান ৩৫ বছর বয়সী পুরুষ গত ২ দিন ধরে জ্বর এবং মাথাব্যথায় ভুগছেন। তাপমাত্রা ১০২ ডিগ্রি। প্যারাসিটামল ৫০০ মিলিগ্রাম দিনে ৩ বার ৩ দিন খেতে দিয়েছি।",
    "রোগী ফাতিমা খাতুন ৪৫ বছর বয়সী মহিলা রক্তচাপ ১৪০/৯০। ডায়াবেটিসের ইতিহাস আছে। ওজন ৬৫ কেজি। মেটফরমিন ৫০০ মিলিগ্রাম দিনে ২ বার।",
    "৮ বছর বয়সী ছেলে শিশুর গত ৩ দিন ধরে জ্বর ও কাশি। পরীক্ষা: রক্ত পরীক্ষা, এক্স রে। ওষুধ: প্যারাসিটামল ২৫০ মিলিগ্রাম দিনে ৩ বার",
    "রোগী জ্বর এবং মাথাব্যথায় ভুগছেন। তাপমাত্রা ১০২ ডিগ্রি। প্যারাসিটামল ৫০০ মিলিগ্রাম দিনে ২ বার ৩ দিন খেতে দিয়েছি।",
    "রক্তচাপ ১৪০/৯০। ডায়াবেটিসের ইতিহাস আছে। মেটফরমিন ৫০০ মিলিগ্রাম দিনে ২ বার।",
    "পেটব্যথা এবং বমি। গত ৩ দিন ধরে সমস্যা। ওমিপ্রাজল ২০ মিলিগ্রাম সকালে খালি পেটে।",
    "৪৫ বছর বয়সী মহিলা ডায়াবেটিস এবং উচ্চরক্তচাপে আক্রান্ত। ইনসুলিন ২০ ইউনিট দিনে ২ বার সকালে ও রাতে খেতে বলা হয়েছে। ওজন ৬৮ কেজি।",
    "৩২ বছর বয়সী পুরুষ পেটব্যথা ও বমির জন্য ভর্তি ছিলেন। অ্যামক্সিসিলিন ৫০০ মিলিগ্রাম দিনে ৩ বার ৭ দিন সেবন করতে হয়েছে। প্রস্রাব পরীক্ষা ও আল্ট্রাসাউন্ড করানো হয়েছে।",
    "৬০ বছর বয়সী পুরুষ হৃদরোগে ভুগছেন। কুইনাপ্রিল ৭৫ মিলিগ্রাম প্রতিদিন। রক্তচাপ ১৫০/১০০ mmHg."
]

print("\nProcessing sample clinical texts:")
print("=" * 50)

for i, text in enumerate(sample_clinical_texts, 1):
    print(f"\nDocument {i}:")
    print(f"Input: {text}")

    # Extract information
    extracted_info = extractor.extract_clinical_information(text)

    # Display key information in the required format
    print(f"Patient Name: {extracted_info.get('patient_name', 'Not specified')}")
    print(f"Age: {extracted_info.get('age', 'Not specified')}")
    print(f"Symptoms: {extracted_info['symptoms']}")
    print(f"Diseases: {extracted_info['diseases']}")
    print(f"Medications: {extracted_info['medications']}")
    print(f"Duration of Condition: {extracted_info.get('duration_of_condition', 'Not specified')}")
    print(f"Diagnostic Tests: {extracted_info['diagnostic_tests']}")
    print(f"Vital Signs: {extracted_info['vital_signs']}")
    print(f"Processing Time: {extracted_info['processing_time']:.3f}s")

# Display system statistics
print(f"\nSystem Statistics:")
stats = extractor.get_stats()
for key, value in stats.items():
    print(f"{key}: {value}")

# ==========================================
# EVALUATION FRAMEWORK DEMO
# ==========================================

print("\n" + "="*50)
print("EVALUATION FRAMEWORK DEMONSTRATION")
print("="*50)

# Sample ground truth for evaluation
ground_truth_entities = {
    'SYMPTOM': ['জ্বর', 'মাথাব্যথা', 'পেটব্যথা', 'বমি', 'কাশি'],
    'DISEASE': ['ডায়াবেটিস', 'হৃদরোগ'],
    'MEDICATION': ['প্যারাসিটামল', 'মেটফরমিন', 'ইনসুলিন', 'এসপিরিন']
}

# Extract entities from first few sample texts for evaluation
predicted_entities = {
    'SYMPTOM': [],
    'DISEASE': [],
    'MEDICATION': []
}

for text in sample_clinical_texts[:4]:  # Use first 4 texts
    entities = recognizer.extract_entities(text)
    predicted_entities['SYMPTOM'].extend([e[0] for e in entities.get('SYMPTOM', [])])
    predicted_entities['DISEASE'].extend([e[0] for e in entities.get('DISEASE', [])])
    predicted_entities['MEDICATION'].extend([e[0] for e in entities.get('MEDICATION', [])])

# Remove duplicates
for key in predicted_entities:
    predicted_entities[key] = list(set(predicted_entities[key]))

# Evaluate
evaluation_results = evaluator.evaluate_entity_extraction(predicted_entities, ground_truth_entities)

print("\nEntity Extraction Evaluation Results:")
for entity_type, metrics in evaluation_results.items():
    if entity_type != 'macro_avg':
        print(f"{entity_type}: P={metrics['precision']:.3f}, R={metrics['recall']:.3f}, F1={metrics['f1']:.3f}")

if 'macro_avg' in evaluation_results:
    macro = evaluation_results['macro_avg']
    print(f"macro_avg: P={macro['precision']:.3f}, R={macro['recall']:.3f}, F1={macro['f1']:.3f}")

print("\nComplete Bangla Clinical Information Extraction Protocol demonstrated successfully!")
print("\n" + "="*70)
print("PROTOCOL SUMMARY")
print("="*70)
print("Text Normalization - Unicode, digit standardization, abbreviation expansion")
print("Clinical Tokenization - Medical compound handling, sentence segmentation")
print("Entity Recognition - Rule-based + dictionary-based extraction")
print("Relation Extraction - Clinical and temporal relationship patterns")
print("Complete Pipeline - End-to-end clinical information structuring")
print("Evaluation Framework - Precision, recall, F1-score metrics")
print("Performance Optimization - Caching, statistics tracking")
print("="*70)

All packages installed and imported successfully!
Initializing Complete Clinical Information Extraction System...
System initialized successfully!

TEXT NORMALIZATION DEMONSTRATION
Original 1: ডাঃ রহমান রাঃ ৩৫ বছর বয়সী পুরুষ জ্বর ও মাথা ব্যথায় ভুগছেন।
Normalized: ডাক্তার রহমান রোগী 35 বছর বয়সী পুরুষ জ্বর ও মাথাব্যথা ভুগছেন
------------------------------
Original 2: তাপমাত্রা ১০২.৫° ফারেনহাইট। পিE: সাধারণ অবস্থা ভাল।
Normalized: তাপমাত্রা 102.5 ফারেনহাইট পি: সাধারণ অবস্থা ভাল
------------------------------
Original 3: এ/পি: প্যারাসিটামল ৫০০ মিগ্র দিনে ৩ বার ৫ দিন।
Normalized: এ/পি: প্যারাসিটামল 500 মিগ্র দিনে 3 বার 5 দিন
------------------------------

TOKENIZATION DEMONSTRATION
Original: রোগী জ্বর ও মাথাব্যথায় ভুগছেন। তাপমাত্রা ১০২.৫ ডিগ্রি। প্যারাসিটামল ৫০০ মিলিগ্রাম দিনে ৩ বার।

Sentences: ['রোগী জ্বর ও মাথাব্যথায় ভুগছেন', 'তাপমাত্রা ১০২.৫ ডিগ্রি', 'প্যারাসিটামল ৫০০ মিলিগ্রাম দিনে ৩ বার']

Tokens: ['রোগী', 'জ্বর', 'ও', 'মাথাব্যথায়', 'ভুগছেন।', 'তাপমাত্রা', '১০২.৫ ডিগ্রি।', 'প্য